Introducing DeReKoGram
======================

Alexander Koplenig

contact: <koplenig@ids-mannheim.de>

2023-03-06

---


### What is this document?

This document accompanies a short paper [[1](#1-wolfer-sascha-koplenig-alexander-kupietz-marc-müller-spitzer-carolin-submitted-introducing-derekogram)]. With this document, we want to demonstrate some basic use cases for the 1- to 3-gram frequency lists provided in the repository of the Leibniz Institute for the German Language (IDS) and make it easier for other researchers to work with the data. 
In this document we are providing **Stata** code (```version 17```) to work with the frequency lists.
##How to use?

* First, you need to download the dataset(s) you want to work with in the 
[IDS repository](https://hdl.handle.net/10932/00-057D-0921-30F0-F201-D)
. The naming convention of the datasets is as follows: ```[1/2/3]-gram-token-lemma-pos-freqs-[with/without]-punctuation.<fold number with leading zero>.tsv.```
* Then unzip the dataset(s) with ```unxz``` on Linux or macOS. On Windows, you can use 
[7-Zip](https://www.7-zip.org/download.html) 
to decompress xz files.

We will demonstrate most of the operations documented here with only a subset of the 16 folds contained in the original data. Please refer to the accompanying publication (Section ‘Evaluation of fold distribution’) to see why, in many cases, it should not make a huge difference whether you use a subset of folds or all folds.

All following sections are ‘self-contained’ in the sense that no code in these sections relies on code in previous sections (e.g. loading of data).


Setup the environment and set the working directory

In [1]:
import os
import csv
from collections import defaultdict
import pandas as pd
import pickle
import math
import random
import numpy as np

os.chdir('/home/koplenig/DeReKoGram')
os.getcwd()

'/home/koplenig/DeReKoGram'

### Decrypting’ integer codes

Wordforms and lemmas in DeReKoGram are integer-coded  [[2](#2-brants-thorsten-popat-ashok-c-xu-peng-och-franz-j-dean-jeffrey-2007-large-language-models-in-machine-translation-in-proceedings-of-the-2007-joint-conference-on-empirical-methods-in-natural-language-processing-and-computational-natural-language-learning-emnlp-conll-858867-prague-czech-republic-association-for-computational-linguistics-retrieved-from-httpsaclanthologyorgd07-1090)]. The information for back-translating or ‘decrypting’ the integer codes is saved in the ```[lemma/token]_keys.<fold number with leading zero>.tsv.xz``` dictionary files. The complete mapping over all folds can be found in ```[lemma/token]_keys.all.tsv.xz```. The files without fold number are the overall dictionaries for the complete dataset. Integer codes are consistent over the single-fold dictionaries and the overall dictionary.

As we wrote in the article, the columns in the unigram datasets are sorted as follows
1: ```Wordform code``` TAB 2: ```Lemma code``` TAB 3: ```POS tag``` TAB 4: ```Frequency```

The first three columns are repeated for bigrams. In this case, the first three columns are repeated for the second element of the bigram, like so:
1: ```Wordform code 1``` TAB 2: ```Lemma code 1``` TAB 3: ```POS tag 1``` TAB 4: ```Wordform code 2``` TAB 5: ```Lemma code 2``` TAB 6: ```POS tag 2``` TAB 7: ```Frequency```

For trigrams, this works accordingly. Consequently, trigram datasets have ```3 * 3 + 1 = 10 columns```.


List the ten most frequent forms (integer code 0-9) with corresponding lemma & wordform information

In [2]:
# First, create we create a dictionary of lemma strings and ids using 
lemma_dict = defaultdict(str)
with open('lemma_keys.all.tsv', 'r') as lemma_file:
    lemma_reader = csv.reader(lemma_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in lemma_reader:
        lemma_string, lemma_id = row
        lemma_dict[lemma_id] = lemma_string

# Next, create a dictionary of token strings and ids
token_dict = defaultdict(str)
with open('token_keys.all.tsv', 'r') as token_file:
    token_reader = csv.reader(token_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in token_reader:
        token_string, token_id = row
        token_dict[token_id] = token_string

# Next, create a list of (token_string, lemma_string, pos, frequency) tuples
fold01 = []
with open('1-gram-token-lemma-pos-freqs-with-punctuation.01.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold01.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))

# Sort the list by frequency in descending order
fold01.sort(key=lambda x: x[3], reverse=True)

List the top 10

In [3]:
fold01[:10]

[('.', '.', '$.', 134870098),
 (',', ',', '$,', 126965247),
 ('der', 'die', 'ART', 64935117),
 ('die', 'die', 'ART', 54882541),
 ('und', 'und', 'KON', 50302414),
 ('"', '"', '$(', 40678679),
 ('in', 'in', 'APPR', 38193220),
 ('den', 'die', 'ART', 25292562),
 (':', ':', '$.', 21626291),
 ('mit', 'mit', 'APPR', 19625776)]

Let us check a few word forms where the lemma should differ from its associated wordform: 

In [4]:
for token, lemma, pos, freq in fold01:
    if token == 'ging' or token == 'sagte' or token == 'Linguistinnen' or token == 'beste':
        print(token, lemma, pos, freq)

sagte sagen VVFIN 1904814
ging gehen VVFIN 667330
beste gut ADJA 192421
Linguistinnen Linguistin NN 13


Of course, we can do this the other way around and search for all wordforms associated with a specific lemma.

In [5]:
for token, lemma, pos, freq in fold01:
    if lemma == 'vormachen':
        print(token, lemma, pos, freq)

vorgemacht vormachen VVPP 3730
vormachen vormachen VVINF 2375
vormacht vormachen VVFIN 505
vorzumachen vormachen VVIZU 335
vormachte vormachen VVFIN 158
vormachten vormachen VVFIN 80
Vorgemacht vormachen VVPP 66
vormachen vormachen VVFIN 49
vormache vormachen VVFIN 46
Vormachen vormachen VVINF 8
vormachst vormachen VVFIN 3
VORGEMACHT vormachen VVPP 1
Vormachen vormachen VVFIN 1


### Aggregating datasets

For DeReKoGram, (at least) two types of aggregations might become relevant:

    1. Aggregating wordform frequencies by lemma and/or POS column. This yields, for example, a dataset with POS token frequencies (one frequency value per POS tag).
    2. Aggregating a dataset that is a combination of multiple folds (for example when compiling frequency information from four folds into one).

We will go into detail below.

#### Aggregation by lemma and/or part-of-speech

We are demonstrating this aggregation with the unigram dataset (fold 08).


First read in fold 08

In [6]:
fold08 = []
with open('1-gram-token-lemma-pos-freqs-with-punctuation.08.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold08.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))
# Sort the list by frequency in descending order
fold08.sort(key=lambda x: x[3], reverse=True)

fold08[:10]

[('.', '.', '$.', 134713792),
 (',', ',', '$,', 126822115),
 ('der', 'die', 'ART', 64888400),
 ('die', 'die', 'ART', 54818658),
 ('und', 'und', 'KON', 50236487),
 ('"', '"', '$(', 40623447),
 ('in', 'in', 'APPR', 38171457),
 ('den', 'die', 'ART', 25282101),
 (':', ':', '$.', 21603035),
 ('mit', 'mit', 'APPR', 19611948)]

List the total number of rows in fold08

In [7]:
'{:,}'.format(len(fold08))

'20,820,567'

How many unique lemmas

In [8]:
'{:,}'.format(len(set([x[1] for x in fold08])))

'1,839,383'

In [9]:
# Compute total token frequency
'{:,}'.format(sum([x[3] for x in fold08]))

'2,694,880,682'

#### Aggregate by lemma and sort by frequency

In [10]:
# Aggregate by lemma
lemma_freqs = defaultdict(int)
for token, lemma, pos, freq in fold08:
    lemma_freqs[lemma] += freq

# Sort by frequency
lemma_freqs = sorted(lemma_freqs.items(), key=lambda x: x[1], reverse=True)

lemma_freqs[:10]

[('die', 235387333),
 ('.', 134795026),
 (',', 126822115),
 ('--', 93049724),
 ('in', 64110264),
 ('und', 52742151),
 ('"', 52045982),
 ('sein', 50240035),
 ('eine', 48215708),
 ('zu', 30443236)]

How many unique values (should be same as above)

In [11]:
'{:,}'.format(len(lemma_freqs))

'1,839,383'

In [12]:
# Assert that lemma_freqs has the same length as the number of unique lemmas in fold08

if len(lemma_freqs) == len(set([x[1] for x in fold08])):
    print('True')
else:
    print('False')    

True


#### Aggegation by lemma AND by POS

In [13]:
lemma_pos_freqs = defaultdict(int)

for token, lemma, pos, freq in fold08:
    lemma_pos_freqs[(lemma, pos)] += freq

lemma_pos_freqs = sorted(lemma_pos_freqs.items(), key=lambda x: x[1], reverse=True)

lemma_pos_freqs[:10]

[(('die', 'ART'), 214882395),
 (('.', '$.'), 134795026),
 ((',', '$,'), 126822115),
 (('--', 'NN'), 54597573),
 (('und', 'KON'), 52742151),
 (('"', '$('), 52045982),
 (('eine', 'ART'), 46955682),
 (('in', 'APPR'), 42021000),
 (('sein', 'VAFIN'), 38290797),
 (('--', 'NE'), 25591553)]

### Combining and aggregating multiple folds
We are now loading three more folds (03, 11 and 15) which we then combine with fold 8.

In [14]:
# Load folds

fold03 = []
with open('1-gram-token-lemma-pos-freqs-with-punctuation.03.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold03.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))

fold11 = []
with open('1-gram-token-lemma-pos-freqs-with-punctuation.11.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold11.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))

fold15 = []
with open('1-gram-token-lemma-pos-freqs-with-punctuation.15.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold15.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))   

Combine all four folds into one dataset

In [15]:
# Combine fold08 with fold03, fold11, and fold15 in a new list
fold08_03_11_15 = fold08 + fold03 + fold11 + fold15

Let's see how many rows there are for periods (end-of-sentence) in the combined dataset.

In [16]:
for token, lemma, pos, freq in fold08_03_11_15:
    if pos == "$." and token == '.':
        print(token, lemma, pos, freq)


. . $. 134713792
. . $. 134745494
. . $. 134816153
. . $. 134982462


Aggregate by token, lemma and pos and sort by frequency

In [17]:
fold08_03_11_15_agg = defaultdict(int)

for token, lemma, pos, freq in fold08_03_11_15:
    fold08_03_11_15_agg[(token, lemma, pos)] += freq

fold08_03_11_15_agg = sorted(fold08_03_11_15_agg.items(), key=lambda x: x[1], reverse=True)

fold08_03_11_15_agg[:10]

[(('.', '.', '$.'), 539257901),
 ((',', ',', '$,'), 507716781),
 (('der', 'die', 'ART'), 259681257),
 (('die', 'die', 'ART'), 219388266),
 (('und', 'und', 'KON'), 201076566),
 (('"', '"', '$('), 162767675),
 (('in', 'in', 'APPR'), 152760271),
 (('den', 'die', 'ART'), 101188094),
 ((':', ':', '$.'), 86507438),
 (('mit', 'mit', 'APPR'), 78477570)]

### Lowering datasets
By "lowering" we mean transforming all wordforms to lower-case (e.g. *Der* and *dEr* are transformed to *der*). 

In [18]:
# Use the ``.lower()`` method to lower-case a string
for token, lemma, pos, freq in fold08:
    if token.lower() == 'befinden' or token.lower() == 'rennen' or token.lower() == 'nato':
        print(token, lemma, pos, freq) 

Rennen Rennen NN 186715
befinden befinden VVFIN 87482
Nato Nato NE 28031
befinden befinden VVINF 6719
NATO NATO NE 5759
rennen rennen VVINF 5424
rennen rennen VVFIN 5094
Befinden Befinden NN 2185
Befinden befinden VVFIN 292
RENNEN Rennen NN 110
naTo -- NN 86
naTo -- ADJA 26
naTo -- VVFIN 18
naTo -- ADJD 14
nato -- ADJA 13
Befinden befinden VVINF 10
nato -- ADJD 3
BEFINDEN Befinden NN 2
BEFINDEN befinden VVFIN 2
naTo Nato NE 2
BeFinden befinden VVFIN 1
NAto -- ADJD 1
NaTo -- NN 1
befiNdeN befinden VVINF 1


### Searching for patterns

In [19]:
# List 10 most frequent lemmas starting with a capital letter that end with 'ung' and starts with in fold08
ung_freq = defaultdict(int)

for token, lemma, pos, freq in fold08:
    if lemma.endswith('ung') and lemma[0].isupper():
        ung_freq[lemma] += freq

ung_freq = sorted(ung_freq.items(), key=lambda x: x[1], reverse=True)

ung_freq[:10]

[('Veranstaltung', 517778),
 ('Entscheidung', 456259),
 ('Ausstellung', 435816),
 ('Regierung', 428677),
 ('Richtung', 422583),
 ('Führung', 391267),
 ('Entwicklung', 384038),
 ('Leistung', 383334),
 ('Zeitung', 358011),
 ('Verfügung', 355676)]

### Cleaning
In Wolfer et al. [1], we show how different cleaning 'stages' of the dataset influence vocabulary growth when taking more and more corpus folds into consideration. Here, we want to demonstrate how this cleaning can be achieved. The cleaning stages reported in the paper are as follows: 

 	A. No cleaning.
 		
 	B. No punctuation, names, start-end-symbols, URLs, wordforms only consisting of numbers
 		
 	C. No wordforms containing any numbers
 		
 	D. No wordforms containing upper-case letters that follow lower-case letters
 		
 	E. Only wordforms where the TreeTagger assigned a lemma
 		
 	F. Only wordforms that are themselves (or the associated lemma) on a basic lemma list of New High German standard language [5].

Cleaning stages A through D are cumulative, e.g. cleaning stage D incorporates stages B and C. Stages E and F, however, both rely on stage D. Here, we only show how to create cleaning stage B based on the original dataset (A) because this already demonstrates the logic behind the process. Cleaning the dataset can be understood as an extension of [searching for specific patterns](#searching-for-patterns), because we first identify relevant codes that we want to exclude and then exclude these codes from the frequency dataset by subsetting. We are demonstrating this using fold 14.


In [20]:
fold14 = []

with open('1-gram-token-lemma-pos-freqs-with-punctuation.14.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        fold14.append((token_dict[token_id],lemma_dict[lemma_id], pos, int(frequency)))

Excluding punctuation, names, start-end-symbols, URLs, wordforms only consisting of numbers

In [21]:
import re
fold14_filtered = []

for token, lemma, pos, freq in fold14:
    if pos != '$.' and pos != '$(' and pos != '$,' and pos != '$)' and pos != 'NE' and not token.isdigit() and not re.match("^http[s]?://", token):
        fold14_filtered.append((token, lemma, pos, freq))

Count the number of tokens in fold14_filtered that are not in fold14

In [22]:
'{:,}'.format(len(fold14) - len(fold14_filtered))

'2,418,152'

### A note on START/END markers

You might want to remove all rows in the dataset that consist of start or end markers only. You can do this by excluding rows that only contain the integer codes ```-1``` or ```-2``` in the wordform and lemma column(s). 

We’ve included these “markers-only rows” for consistency checks, for example to check whether the frequencies (in the bigram case) for ```«START»``` ```«START»``` and ```«START»``` ```<other>``` match. In the unigram datasets, the number of markers can also be used to find out the number of documents that went into a corpus fold because each ```«START»``` marks the beginning of a document and each ```«END»``` the end of a document For simple frequency tables, the “markers-only rows” are not necessary and might even distort some measures like normed/relative frequencies.

However, there are situations where keeping such rows can be important. For example, when we compute  $P_{ml} =\frac{c(v,w)}{c(v)}$ in the following section based on the ```2-gram data```, we can use the ```1-gram data``` to get the information for the denominator $c(v)$. If we exclude the markers-only rows from the ```1-gram data```, this would lead to a denominator of 0 for v equal to ```«START»``` and thus an undefined value.

When we read in the ```3-gram data``` below, we exclude the markers-only rows.

### Train smoothed *n*-gram models

Here, we train a smoothed n-gram models of varying order. Probabilities are smoothed with a technique called Witten-Belll smoothing [6].

We start with a smoothed 1-gram model

The smoothed 1-gram probability can be calculated as follows:


$P_{sm} (w)=λ_Κ P_{ml} (w)+ (1-λ_Κ)\frac{1}{Κ}$

where $P_{ml} (w)$ is the maximum likelihood estimate of the probability of word $w$ computed as follows:

$P_{ml} (w)=\frac{C(w)}{N}$

where $c(w)$ is the number of times word $w$ occurs in the corpus and $N$ is the total number of tokens in the corpus. The discounting factor $λ_Κ$ is calculated as follows:

$ (1-λ_Κ) = \frac{1}{N+K}$

where $K$ is the vocabulary of the corpus. Thus, $P_{sm} (w)$ can be calculated as follows:

$P_{sm} (w)=\frac{C(w)+1}{N+K}$


Let's compute the distribution of smoothed 1-gram probabilities for fold08.

First we need to load fold08. But we don't replace token ids with actual tokens from the token_dict, because we want to keep the token ids for the next step

In [63]:
fold08_1gram_agg = defaultdict(int)

with open('1-gram-token-lemma-pos-freqs-with-punctuation.08.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id, lemma_id, pos, frequency = row
        # Aggregate frequencies for each token id 
        fold08_1gram_agg[token_id] += int(frequency)

Now we can use 'fold08_1gram_agg' to calculate p_1gram

In [64]:
# Compute lambda_k

# calculate the total number of tokens in fold08
total_tokens = sum(fold08_1gram_agg.values())

# Calculate the number of unique tokens in fold08
unique_tokens = len(fold08_1gram_agg)

# Store both in a pickle file for later use

with open('total_tokens_and_unique_tokens.pkl', 'wb') as pickle_file:
    pickle.dump((total_tokens, unique_tokens), pickle_file)   

In [65]:
# Now generate the p_1gram dictionary
p_1gram = {}

for token_id, freq in fold08_1gram_agg.items():
    p_1gram[token_id] = (freq) / (total_tokens)

# Store 'p_1gram' in a pickle file for later use

with open('p_sm_1gram.pickle', 'wb') as p_1gram_file:
    pickle.dump(p_1gram, p_1gram_file)

# List first 10 items in 'p_1gram'
list(p_1gram.items())[:10]

[('0', 0.04999791489840811),
 ('1', 0.04706038224515351),
 ('2', 0.02531099742396684),
 ('3', 0.022993389063152594),
 ('4', 0.018641451302666617),
 ('5', 0.01507430264773407),
 ('6', 0.01416443305076911),
 ('7', 0.009511099386032113),
 ('8', 0.008016323373533314),
 ('9', 0.00752293529558204)]

Next we import "token_keys.08.tsv" that contains the token ids and the actual tokens. 

In the first column we have the actual tokens, in the second column we have the token ids.
We need to create a dictionary that maps token ids to actual tokens


In [66]:
token_keys08 = {}

with open('token_keys.08.tsv', 'r') as token_keys_file:
    token_keys_reader = csv.reader(token_keys_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in token_keys_reader:
        token, token_id = row
        token_keys08[token_id] = token

# List first 10 items in token_keys08
list(token_keys08.items())[:10]

[('-2', '«END»'),
 ('-1', '«START»'),
 ('0', '.'),
 ('1', ','),
 ('2', 'der'),
 ('3', 'die'),
 ('4', 'und'),
 ('5', '"'),
 ('6', 'in'),
 ('7', 'den')]

In [67]:
# Now prepare a list of ten token ids randomly selected from fold08_1gram_agg

import random

random.seed(42)

ten_random_token_ids = []

for i in range(10):
    ten_random_token_ids.append(random.sample(list(fold08_1gram_agg.keys()), 10))

# List the ten random token ids horizontally

for ten_random_token_ids in ten_random_token_ids:
    print(*ten_random_token_ids, sep=' ')

5789201 844038 9438987 52266847 9875656 3856955 10681738 45880705 2653540 19035560
1197905 798548 2452324 5549296 49975452 97778959 891982 20045355 4112821 45861651
5895077 24418599 5174075 113776396 33898514 222617 9974381 28245822 68823919 14319911
17869898 11859698 36508282 3744357 3717872 27440269 3139531 16318516 18454807 33286538
1395181 11653453 4737733 5244309 13416537 3150565 106465647 34578141 9094561 6838996
28587208 2652935 1823515 31642671 10580183 2011761 50004445 6153838 9589400 58011328
41772236 37785112 4200273 13388176 26969831 4301734 12040873 2112875 3617345 30353477
20505049 7119568 90266415 75706738 6475659 107507793 4310080 3656514 1521044 12920044
1212325 18243134 27841156 9944781 1803334 3191876 109521791 21319448 7279804 6800373
14807165 89679326 5304644 33302396 3677132 32486295 46566310 8876414 55372519 6842067


In [68]:
# In the above list, replace the token ids with actual tokens

random.seed(42)

ten_random_token_ids = []

for i in range(10):
    ten_random_token_ids.append(random.sample(list(fold08_1gram_agg.keys()), 10))

for ten_random_token_ids in ten_random_token_ids:
    print(*[token_keys08[token_id] for token_id in ten_random_token_ids], sep=' ')

Untersuchungserfolg begreifenden Blechschirm 984193 034347/61 Tempogesellschaft Götz-Herrmann einenÜberraschungssieger Exklusivvertretung Notkunden
8425 Klenower Tränenliste säkular-nationalen 2015GAnfang UngarischeMalerei 81-84 konnte.Höhepunkt Seung-hee eckert@wz.de
Neutralitätsgedanke Österreicher-Vereinigung Religionskrise äVerhandeln Brausesackerln Kellergassen CeMAT-Präsidiums Nächtigungen.Im GreizEmma Brandschutzwochenende
Barempfang hochspezifizierte Gewinner-Lösung Glühtee Smog-Tage Konzernsaal 21.45Tatort Ida-Helene HDMI-2.0-Buchsen Baecker-Handwerk
Messröhrchen Saison-Klassierung artuntypisch Gerichtsübersetzer Kombi-Turniere Klassiktitel gerüstetGGreiz Daumennagel-großer Intensivstation-Patienten tarockierte
Promi-Neugier Döhlauer Sek-Lehrer 110-kV-Oberleitung Country-Laune Musikerheuriger 2052,75 Fakultätsgutachten Kopf-Sprung Braten-Muffel
Rosen-Schaubeete Jetzt-zeigen-wir Dröhn- Kanzlerinnen-Neuwahl Hornnagel Werkhausgasse Bauernkastln Levorato Grattagen abzuschließen.br

In [69]:
# Now we repeat the above but this time, we draw tokens according to pm_sm_1gram

def draw_random_tokens(p_1gram, total_tokens, unique_tokens, num_samples):
    lambda_k = 1 / (total_tokens + unique_tokens)
    weighted_probs = [(1-lambda_k) * p + lambda_k/unique_tokens for p in p_1gram.values()]
    tokens = list(p_1gram.keys())
    samples = random.choices(tokens, weights=weighted_probs, k=num_samples)
    return samples

num_samples = 10
num_tokens = 10

for i in range(num_samples):
    random_tokens = draw_random_tokens(p_1gram, total_tokens, unique_tokens, num_tokens)
    print(*[token_keys08[token_id] for token_id in random_tokens], sep=' ')

hatte lang er Kolsterbrunnen die diese Front defensiven und und
ums bleiben Uhr machte will ( Trainer ergriffenen Hamburger der
Kahlschlag Thorsten erledigt als nicht und . xyxHTMLyxy setzten Krulis
Evangelischen Aber der Geburtstag Grasser Aber den ' und Croy
Halbwahrheiten drückend Hilfe neuen der wird Forschungsbefunde Vielen für für
Schüler , geprägt Damm losschlägt lassen die hielt Villmarer "
sollte erzählt stampfend Wartungs- Favoriten meinem Freundin vor Ideen '
strotzt ihre sich , . Haus Projekt . 16.30 ,
, . «END» gestern das Zeit Woche Weltmeister Parkplatzes hinter
für muss zur , Mensch aus wie kosteten , bevor


In [70]:
# Let's compute the probability of the sentence 'Wir arbeiten am Institut für Deutsche Sprache .'

# We need a program that takes a sentence and returns a list of token ids based on the token_keys08 dictionary
# If a token is not in the dictionary, we assign it the id -777

def sentence_to_token_ids(sentence):
    token_ids = []
    for token in sentence.split():
        try:
            id = list(token_keys08.keys())[list(token_keys08.values()).index(token)]
        except ValueError:
            id = -777
        token_ids.append(id)
    return token_ids

sentence_to_token_ids('Wir arbeiten am Institut für Deutsche Sprache .')


['100', '730', '30', '2438', '15', '662', '1467', '0']

In [71]:
# Now we use the sentence_to_token_ids function to compute the probability of the sentence 'Wir arbeiten am Institut für Deutsche Sprache .'

def sentence_probability(sentence, p_1gram, total_tokens, unique_tokens):
    token_ids = sentence_to_token_ids(sentence)
    lambda_k = 1 / (total_tokens + unique_tokens)
    prob = 1
    for i in range(len(token_ids)):
        w = token_ids[i]
        if w in p_1gram:
            p = (1-lambda_k) * p_1gram[w] + lambda_k/unique_tokens
        else:
            p = lambda_k/unique_tokens
        prob *= p
    return prob

sentence = 'Wir arbeiten am Institut für Deutsche Sprache .'
sentence_prob = sentence_probability(sentence, p_1gram, total_tokens, unique_tokens)
print(f"Probability of sentence '{sentence}': {sentence_prob}")


Probability of sentence 'Wir arbeiten am Institut für Deutsche Sprache .': 2.2529093562602777e-26


In [72]:
# Let's try some gibberish

# Find token ids for 'gibberish'

sentence_to_token_ids('gibberish')



[-777]

In [73]:
# Since this is not available in the dictionary, we assign it the id -777.
# What is the smoothed probability of the sentence 'gibberish'

sentence = 'gibberish'
sentence_prob = sentence_probability(sentence, p_1gram, total_tokens, unique_tokens)
print(f"Probability of sentence '{sentence}': {sentence_prob}")

Probability of sentence 'gibberish': 1.8601846398709812e-17


Compare this to
$ \frac{1}{N+K} \frac{1}{K}$


In [74]:
print(1/(total_tokens + unique_tokens)*1/unique_tokens) 

1.8601846398709812e-17


In [75]:
# Let's compute the probability of the sentence 'Das ist ein richtiger Satz .'

sent1 = 'Das ist ein richtiger Satz .'
sent1_prob = sentence_probability(sent1, p_1gram, total_tokens, unique_tokens)
print(f"Probability of sentence '{sent1}': {sent1_prob:.2e}")

Probability of sentence 'Das ist ein richtiger Satz .': 1.18e-18


In [76]:
# What about an ungrammatical sentence, e.g. 'Das ist ein richtige Satz .'

sent2 = 'Das ist ein richtige Satz .'
sent2_prob = sentence_probability(sent2, p_1gram, total_tokens, unique_tokens)
print(f"Probability of sentence '{sent2}': {sent2_prob:.2e}")

Probability of sentence 'Das ist ein richtige Satz .': 1.16e-17


In [77]:
# Compute ratio of probabilities

ratio = sent2_prob / sent1_prob

# Print the ratio

print('"Das ist ein richtige Satz ." is {:.2f} times more probable than "Das ist ein richtiger Satz ."'.format(ratio))

"Das ist ein richtige Satz ." is 9.85 times more probable than "Das ist ein richtiger Satz ."


"Das ist ein richtige Satz ." is 9.85 times more probable than "Das ist ein richtiger Satz ."
This is because our model only is a so called 'bag-of-words' model, i.e. it does not take into account the order of words. 'richtiger' is simply more frequent than 'richtige' in the corpus.

In [78]:
# Compute the probability of 'richtiger' and 'richtige'

p_richtiger = sentence_probability('richtiger', p_1gram, total_tokens, unique_tokens)
p_richtige = sentence_probability('richtige', p_1gram, total_tokens, unique_tokens)

# Print

print(f'Probability of "richtiger": {p_richtiger:.5f}')
print(f'Probability of "richtige": {p_richtige:.5f}')

Probability of "richtiger": 0.00001
Probability of "richtige": 0.00005


Therefore:

In [79]:
'{:.2f}'.format(p_richtige/p_richtiger)

'9.85'

Thus, we should be able to improve our model by incorporating the context of the word. Let's train a smoothed 2-gram model.
The smoothed conditional probability of w given v can be calculated as:

$P_{sm} (w│v)=λ_v P_{ml} (w│v)+(1-λ_v)P_{sm} (w)$

where $P_{ml} (w│v)$ is the maximum likelihood estimate of the probability of word $w$ given word $v$ and where $λ_v=\frac{C(v)}{C(v)+γ_v }$ is calculated based on $γ_v$ that denotes the number of different words observed after $v$. 

Load the 2-gram frequencies for fold 08 consisting of ```token id1, lemma id1, pos1, token id2, lemma id2, pos2, frequency```

We only need ```token id1, token id2, and frequency```

In [80]:
fold08_2gram_agg = defaultdict(int)

with open('2-gram-token-lemma-pos-freqs-with-punctuation.08.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id1, lemma_id1, pos1, token_id2, lemma_id2, pos2, frequency = row
        # Aggregate the frequency by token_id1 and token_id2
        fold08_2gram_agg[(token_id1, token_id2)] += int(frequency)

# Store the aggregated 2-gram frequencies in a pickle file
print('Storing aggregated 2-gram frequencies in fold08_2gram_agg.pkl')
with open('fold08_2gram_agg.pkl', 'wb') as f:
    pickle.dump(fold08_2gram_agg, f)

Storing aggregated 2-gram frequencies in fold08_2gram_agg.pkl


In [81]:
# List the 10 first entries of fold08_2gram_agg

list(fold08_2gram_agg.items())[:10]

[(('0', '20'), 10394998),
 (('0', '5'), 9403062),
 (('-2', '-2'), 9107988),
 (('-1', '-1'), 9107988),
 (('6', '2'), 8437829),
 (('1', '39'), 6867551),
 (('1', '3'), 9999401),
 (('5', '1'), 6305548),
 (('0', '33'), 5784250),
 (('0', '-2'), 4496264)]

Compute $P_{ml} =\frac{c(v,w)}{c(v)}$

In [82]:
def compute_conditional_probs(fold08_2gram_agg, fold08_1gram_agg):
    p_2gram = {}
    for (v, w), freq_wv in fold08_2gram_agg.items():
        freq_v = fold08_1gram_agg[v]
        p_v_given_w = freq_wv / freq_v
        if v not in p_2gram:
            p_2gram[v] = {}
        p_2gram[v][w] = p_v_given_w
    
    for v in p_2gram.keys():
        total = sum(p_2gram[v].values())
        for w in p_2gram[v].keys():
            p_2gram[v][w] /= total
    
    return p_2gram

p_2gram = compute_conditional_probs(fold08_2gram_agg, fold08_1gram_agg)

# Store the 2-gram probabilities in a pickle file
print('Storing 2-gram probabilities in p_2gram.pkl')
with open('p_2gram.pkl', 'wb') as f:
    pickle.dump(p_2gram, f)

Storing 2-gram probabilities in p_2gram.pkl


Now we need to compute weights $λ_v=\frac{C(v)}{C(v)+γ_v }$ that are calculated based on $γ_v$ that denotes the number of different words observed after $v$.  We can do this by using the 'fold08_2gram_agg' dataset.

In [83]:
def compute_weights(fold08_2gram_agg, fold08_1gram_agg):
    weights = {}
    for (v, w), freq_wv in fold08_2gram_agg.items():
        if v not in weights:
            weights[v] = 0
        weights[v] += 1
        
    for v, num_w in weights.items():
        freq_v = fold08_1gram_agg[v]
        weights[v] = freq_v / (freq_v + num_w)
    
    return weights

lambdas_2gram = compute_weights(fold08_2gram_agg, fold08_1gram_agg)

# Store the weights in a pickle file
print('Storing weights in lambdas_2gram.pkl')
with open('lambdas_2gram.pkl', 'wb') as f:
    pickle.dump(lambdas_2gram, f)

Storing weights in lambdas_2gram.pkl


In [84]:
# List the 10 first entries of lambdas_2gram

list(lambdas_2gram.items())[:10]

[('0', 0.9877858773490689),
 ('-2', 0.9999998902062793),
 ('-1', 0.9168784375436645),
 ('6', 0.9875822418016662),
 ('1', 0.9870102987693855),
 ('5', 0.9648158064744667),
 ('15', 0.9718534878399419),
 ('19', 0.9790236724640878),
 ('34', 0.9874086259889072),
 ('8', 0.9617792749746221)]

In [85]:
# List entries of p_2gram for 'weinerlichem'

token = 'weinerlichem'
token_id = list(token_keys08.keys())[list(token_keys08.values()).index(token)]
p_2gram[token_id]

{'67932': 0.20000000000000004,
 '3209': 0.16666666666666669,
 '514': 0.13333333333333336,
 '20061': 0.06666666666666668,
 '30860': 0.06666666666666668,
 '1503': 0.03333333333333334,
 '131807': 0.03333333333333334,
 '4434': 0.03333333333333334,
 '2974': 0.03333333333333334,
 '181154': 0.03333333333333334,
 '930477': 0.03333333333333334,
 '2198576': 0.03333333333333334,
 '49500': 0.03333333333333334,
 '38939': 0.03333333333333334,
 '5951': 0.03333333333333334,
 '3087': 0.03333333333333334}

In [86]:
# Check count of 'weinerlichem'

fold08_1gram_agg[token_id]

30

In [87]:
# With that, we should be able to compute the lambda of 'weinerlichem'

# 16 different tokens follow 'weinerlichem' in the corpus

len(p_2gram[token_id])

# Compute lambda

fold08_1gram_agg[token_id] / (fold08_1gram_agg[token_id] + len(p_2gram[token_id]))

0.6521739130434783

In [88]:
# Check the compute weight of 'weinerlichem'

lambdas_2gram[token_id]

assert lambdas_2gram[token_id] == fold08_1gram_agg[token_id] / (fold08_1gram_agg[token_id] + len(p_2gram[token_id]))

Now we we can draw tokens according to 'pm_sm_2gram' (note that we still use 'pm_sm_1gram' to draw the first token)

In [89]:
# First we pre-compute smoothed probabilities for the 1-gram model and store them in a dictionary

p_1gram_smoothed = {}

for token_id, freq in fold08_1gram_agg.items():
    p_1gram_smoothed[token_id] = (freq + 1) / (total_tokens + unique_tokens)

# Store the smoothed 1-gram probabilities in a pickle file

print('Storing smoothed 1-gram probabilities in p_1gram_smoothed.pkl')

with open('p_1gram_smoothed.pkl', 'wb') as f:
    pickle.dump(p_1gram_smoothed, f)

Storing smoothed 1-gram probabilities in p_1gram_smoothed.pkl


In [90]:
def draw_random_tokens(p_1gram_smoothed, p_2gram, lambdas_2gram, num_tokens=10):
    tokens = []
    
    # Draw the first token from the 1-gram model
    first_token = random.choices(list(p_1gram_smoothed.keys()), weights=list(p_1gram_smoothed.values()), k=1)[0]
    tokens.append(first_token)
    
    # Draw the remaining tokens from the 2-gram model
    for i in range(1, num_tokens):
        last_token = tokens[-1]
        if last_token in p_2gram:
            cond_probs = {token: lambdas_2gram[last_token] * p_2gram[last_token][token] + (1 - lambdas_2gram[last_token]) * p_1gram_smoothed[token] for token in p_2gram[last_token]}
            next_token = random.choices(list(cond_probs.keys()), weights=list(cond_probs.values()), k=1)[0]
            tokens.append(next_token)
        else:
            tokens.append(random.choices(list(p_1gram_smoothed.keys()), weights=list(p_1gram_smoothed.values()), k=1)[0])
    
    return tokens

In [91]:
num_samples = 10
num_tokens = 20

for i in range(num_samples):
    random_tokens = draw_random_tokens(p_1gram_smoothed, p_2gram, lambdas_2gram, num_tokens)
    print(*[token_keys08[token_id] for token_id in random_tokens], sep=' ')

Helmut Kohl musste um die Lage auf die bei einer Agrarwende , Kraft erfordern . Alle kamen die von 2,05
ist nicht unbedingt verschwinden lasse sich die keine Randerscheinungen Marmagener CDU-Landtagsabgeordnete Reinhold Gebhardt Wolfgang Steinmetz am Mittwoch , die 2010
, das Land ? Gino Pace , wurde 1990 , die Schützenkönigin war aussichtslos : Kein Wunder , heute gegen
zu der Natur- und wie sie wollen wir drüber fuhren die Leiter des Lebens ( ‘ Es war . Das
des als Stürmer auf Wiedergewinnung alter Wunsch aus - Zeitsoldat bei WM-Endrunden . Wochenlang tot . Und dann ein erstklassiges
TV-Foto : „ Königshaus . Ob Omas und später auch Pokalfinalist Eintracht am häufigsten Erreger in Pentz startet die in
zum Wechsel im Rahmen des Beförderungsbudgets für die Liste / Bratislava ) sowie altersgerechtes Vereinstraining dienstags 13 Uhr , Gott
" Er war geschlagen worden . Viele Leistungsträger in der Furcht vor allem an . Freispruch , fuhren abends die
starken Offensive gegen die mutigen Ents

Now let's compute the probality of the sentences '"Das ein ist richtige Satz ." and "Das ist ein richtiger Satz ." using our smoothed 2-gram model.

In [92]:
# Use sentence_to_token_ids() to convert the sentences to token IDs

# Define a program 'compute_probability_of_sentence' that computes the probability of a sentence

def compute_probability_of_sentence(sentence, p_1gram_smoothed, p_2gram, lambdas_2gram):
    tokens = sentence_to_token_ids(sentence)
    prob = 1
    # Draw the first token from the 1-gram model
    first_token = tokens[0]
    prob *= p_1gram_smoothed[first_token]
    # Draw the remaining tokens from the 2-gram model
    for i in range(1, len(tokens)):
        last_token = tokens[i - 1]
        token = tokens[i]
        if last_token in p_2gram and token in p_2gram[last_token]:
            prob *= lambdas_2gram[last_token] * p_2gram[last_token][token] + (1 - lambdas_2gram[last_token]) * p_1gram_smoothed[token]
        else:
            prob *= p_1gram_smoothed[token]
    return prob


In [93]:
sent1 = compute_probability_of_sentence('Das ist ein richtige Satz .', p_1gram_smoothed, p_2gram, lambdas_2gram)
print('The probability of "Das ist ein richtige Satz ." is {:.2e}'.format(sent1))

The probability of "Das ist ein richtige Satz ." is 8.13e-17


In [94]:
sent2 = compute_probability_of_sentence('Das ist ein richtiger Satz .', p_1gram_smoothed, p_2gram, lambdas_2gram)
print('The probability of "Das ist ein richtiger Satz ." is {:.2e}'.format(sent2))

The probability of "Das ist ein richtiger Satz ." is 2.22e-13


In [95]:
print('"Das ist ein richtiger Satz ." is around {:.0f} times more probable than "Das ist ein richtige Satz ."'.format(sent2/sent1))

"Das ist ein richtiger Satz ." is around 2724 times more probable than "Das ist ein richtige Satz ."


Let's look at the conditional probabilities

In [96]:
print ('p(richtige|ein)',lambdas_2gram[sentence_to_token_ids('ein')[0]] * p_2gram[sentence_to_token_ids('ein')[0]][sentence_to_token_ids('richtige')[0]] + (1 - lambdas_2gram[sentence_to_token_ids('ein')[0]]) * p_1gram_smoothed[sentence_to_token_ids('richtige')[0]])
print ('p(Satz|richtige)',lambdas_2gram[sentence_to_token_ids('richtige')[0]] * p_2gram[sentence_to_token_ids('richtige')[0]][sentence_to_token_ids('Satz')[0]] + (1 - lambdas_2gram[sentence_to_token_ids('richtige')[0]]) * p_1gram_smoothed[sentence_to_token_ids('Satz')[0]])


print ('p(richtiger|ein)',lambdas_2gram[sentence_to_token_ids('ein')[0]] * p_2gram[sentence_to_token_ids('ein')[0]][sentence_to_token_ids('richtiger')[0]] + (1 - lambdas_2gram[sentence_to_token_ids('ein')[0]]) * p_1gram_smoothed[sentence_to_token_ids('richtiger')[0]])
print ('p(Satz|richtiger)',lambdas_2gram[sentence_to_token_ids('richtiger')[0]] * p_2gram[sentence_to_token_ids('richtiger')[0]][sentence_to_token_ids('Satz')[0]] + (1 - lambdas_2gram[sentence_to_token_ids('richtiger')[0]]) * p_1gram_smoothed[sentence_to_token_ids('Satz')[0]])

p(richtige|ein) 3.19724849917519e-06
p(Satz|richtige) 6.403467638530913e-05
p(richtiger|ein) 0.0004839062838288913
p(Satz|richtiger) 0.001152413788250634


Let's continue with a 3-gram model, by computing

$P_{sm} (w│u,v)=\frac{c(uvw)+γ_{u,v} P_{sm} (w│u,v)}{C(u,v)+γ_{u,v}}$

Load the 3-gram frequencies for fold 08 consisting of ````token id1, lemma id1, pos1, token id2, lemma id2, pos2, token id3, lemma id3, pos3, frequency````

We only need ````token id1, token id2, token id3, and frequency````

In [97]:
fold08_3gram_agg = defaultdict(int)

with open('3-gram-token-lemma-pos-freqs-with-punctuation.08.tsv', 'r') as freq_file:
    freq_reader = csv.reader(freq_file, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in freq_reader:
        token_id1, lemma_id1, pos1, token_id2, lemma_id2, pos2, token_id3, lemma_id3, pos3, frequency = row
        # Here, rows where token_id1, token_id2 and token_id3 are -1 or -2  can be removed
        if not (token_id1 in ['-1', '-2'] and token_id2 in ['-1', '-2'] and token_id3 in ['-1', '-2']):
            # Only append token_id1, token_id2, token_id3, and frequency
            fold08_3gram_agg[(token_id1, token_id2, token_id3)] += int(frequency)

# Store the aggregated 3-gram frequencies in a pickle file
print('Storing aggregated 3-gram frequencies in fold08_3gram_agg.pkl')
with open('fold08_3gram_agg.pkl', 'wb') as f:
    pickle.dump(fold08_3gram_agg, f)


Storing aggregated 3-gram frequencies in fold08_3gram_agg.pkl


$P_{sm} (w│u,v)=λ_v P_{ml} (w│u,v)+(1-λ_v)P_{sm} (w|v)$

In [98]:
def compute_conditional_probs(fold08_3gram_agg, fold08_2gram_agg):
    p_3gram = {}

    for (u, v, w), freq_uwv in fold08_3gram_agg.items():
        freq_uv = fold08_2gram_agg[(u, v)]
        p_w_given_uv = freq_uwv / freq_uv
        if (u, v) not in p_3gram:
            p_3gram[(u, v)] = {}
        p_3gram[(u, v)][w] = p_w_given_uv

    for (u, v), p_w_given_uv in p_3gram.items():
        total = sum(p_3gram[(u, v)].values())
        for w in p_3gram[(u, v)]:
            p_3gram[(u, v)][w] /= total

    return p_3gram

p_3gram = compute_conditional_probs(fold08_3gram_agg, fold08_2gram_agg)

# Store the 3-gram probabilities in a pickle file
print('Storing 3-gram probabilities in p_3gram.pkl')
with open('p_3gram.pkl', 'wb') as f:
    pickle.dump(p_3gram, f)       

Storing 3-gram probabilities in p_3gram.pkl


In [99]:
def compute_weights(fold08_3gram_agg, fold08_2gram_agg, fold08_1gram_agg):
    weights = {}
    for (v_1, v_2, w), freq_wv in fold08_3gram_agg.items():
        if (v_1, v_2) not in weights:
            weights[(v_1, v_2)] = 0
        weights[(v_1, v_2)] += 1

    for (v_1, v_2), num_w in weights.items():
        freq_v = fold08_2gram_agg[(v_1, v_2)]
        weights[(v_1, v_2)] = freq_v / (freq_v + num_w)
    
    return weights

lambdas_3gram = compute_weights(fold08_3gram_agg, fold08_2gram_agg, fold08_1gram_agg)

# Store the weights in a pickle file
print('Storing weights in lambdas_3gram.pkl')
with open('lambdas_3gram.pkl', 'wb') as f:
    pickle.dump(lambdas_3gram, f)

Storing weights in lambdas_3gram.pkl


In [100]:
def draw_random_tokens(p_1gram_smoothed, p_2gram, p_3gram, lambdas_2gram, lambdas_3gram, num_tokens=10):
    tokens = []
    
    # Draw the first token from the smoothed unigram probability distribution
    first_token = random.choices(list(p_1gram_smoothed.keys()), weights=list(p_1gram_smoothed.values()), k=1)[0]
    tokens.append(first_token)
    
    # Draw the second token from the smoothed bigram probability distribution
    last_token = tokens[-1]
    cond_probs = {token: lambdas_2gram[last_token] * p_2gram[last_token][token] + (1 - lambdas_2gram[last_token]) * p_1gram_smoothed[token] for token in p_2gram[last_token]}
    next_token = random.choices(list(cond_probs.keys()), weights=list(cond_probs.values()), k=1)[0]
    tokens.append(next_token)
    
    # Draw the remaining tokens from the smoothed trigram probability distribution
    for i in range(2, num_tokens):
        last_two_tokens = (tokens[-2], tokens[-1])
        if last_two_tokens in p_3gram:
            cond_probs = {token: lambdas_3gram[last_two_tokens] * p_3gram[last_two_tokens][token] + (1 - lambdas_3gram[last_two_tokens]) * p_2gram[last_two_tokens[1]][token] for token in p_3gram[last_two_tokens]}
            next_token = random.choices(list(cond_probs.keys()), weights=list(cond_probs.values()), k=1)[0]
            tokens.append(next_token)
        else:
            cond_probs = {token: lambdas_2gram[last_token] * p_2gram[last_token][token] + (1 - lambdas_2gram[last_token]) * p_1gram_smoothed[token] for token in p_2gram[last_token]}
            next_token = random.choices(list(cond_probs.keys()), weights=list(cond_probs.values()), k=1)[0]
            tokens.append(next_token)
    
    return tokens


In [101]:
num_samples = 10
num_tokens = 20

for i in range(num_samples):
    random_tokens = draw_random_tokens(p_1gram_smoothed, p_2gram, p_3gram, lambdas_2gram, lambdas_3gram, num_tokens=num_tokens)
    print(*[token_keys08[token_id] for token_id in random_tokens], sep=' ')

Gesicht der einheimischen Bürger , bei dem Text " Auf die Juristen in der L-Dressur den Sieg des bereits sehr
Geöffnet Die Schlossweihnacht hat Mo-Fr von 15 bis 17.15 Uhr ) steigt das Risiko fu r Humor , Seifenblasen-Wölkchen statt
erfolgreichen gemeinsamen Bemühungen um das BIP sank von 18,4 Prozent liegt der nächste Premierminister wird . Die Verträge für ein
zu vertreiben . Vergeblich mühten sich Nachwuchsfußballer aus der Lüftungsanlage der Parkgarage bezeichnen . Klar ist , mit 6 :
an Flüssigkeit und eine kleinere Rolle . Mit einem schweren Unfall ein " complete pictoral version " ist es vorgekommen
ein 0 : 1. Anne Rudolf vom Weingrosshändler Schlumberger im RP Shop für Reitsportfans der näheren Zukunft zugunsten des Senders
während seiner Amtszeit ( Juni ) , an die Vergangenheit Günter Kommol zeigte Ehemaligen aus den diesjährigen Wettbewerb organisiert .
, entfalte sich die Verwaltung kaum dokumentieren . Doch der TAZ beisteuern ( 2015 ) nahm , gilt als wichtiger
Leute beziehen 

Again let's compute the probality of the sentences '"Das ein ist richtige Satz ." and "Das ist ein richtiger Satz .". Now we use our smoothed 3-gram model.

In [102]:
def compute_probability_of_sentence(sentence, p_1gram_smoothed, p_2gram, p_3gram, lambdas_2gram, lambdas_3gram):
    tokens = sentence_to_token_ids(sentence)
    prob = 1
    # Draw the first token from the 1-gram model
    first_token = tokens[0]
    prob *= p_1gram_smoothed[first_token]
    
    # Draw the second token from the 2-gram model
    if len(tokens) > 1:
        second_token = tokens[1]
        last_token = tokens[0]
        if last_token in p_2gram and second_token in p_2gram[last_token]:
            prob *= lambdas_2gram[last_token] * p_2gram[last_token][second_token] + (1 - lambdas_2gram[last_token]) * p_1gram_smoothed[second_token]
        else:
            prob *= p_1gram_smoothed[second_token]
    
    # Draw the remaining tokens from the 3-gram model
    for i in range(2, len(tokens)):
        last_2_tokens = (tokens[i-2], tokens[i-1])
        token = tokens[i]
        if last_2_tokens in p_3gram and token in p_3gram[last_2_tokens]:
            prob *= lambdas_3gram[last_2_tokens] * p_3gram[last_2_tokens][token] + (1 - lambdas_3gram[last_2_tokens]) * p_2gram[last_2_tokens[1]][token]
        else:
            prob *= p_2gram[last_2_tokens[1]][token] if last_2_tokens[1] in p_2gram and token in p_2gram[last_2_tokens[1]] else p_1gram_smoothed[token]
    
    return prob


In [103]:
sent1 = compute_probability_of_sentence('Das ist ein richtiger Satz .', p_1gram_smoothed, p_2gram, p_3gram, lambdas_2gram, lambdas_3gram)
print('The probability of "Das ist ein richtiger Satz ." is {:.2e}'.format(sent1))   

The probability of "Das ist ein richtiger Satz ." is 7.45e-12


In [104]:
sent2 = compute_probability_of_sentence('Das ist ein richtige Satz .', p_1gram_smoothed, p_2gram, p_3gram, lambdas_2gram, lambdas_3gram)
print('The probability of "Das ist ein richtige Satz ." is {:.2e}'.format(sent2))

The probability of "Das ist ein richtige Satz ." is 1.23e-15


Check if there's a 3-gram "ein", "richtiger" and "Satz" in p_3gram

In [105]:
sentence = 'ein richtiger Satz'
tokens = sentence_to_token_ids(sentence)

# Check if there's a 3-gram "ein", "richtige" and "Satz" in p_3gram
last_two_tokens = (tokens[0], tokens[1])
next_token = tokens[2]

p_3gram[last_two_tokens][next_token] if last_two_tokens in p_3gram and next_token in p_3gram[last_two_tokens] else 0

0.0012951503813498188

Check if there's a 3-gram "ein", "richtige" and "Satz" in p_3gram


In [106]:
sentence = 'ein richtige Satz'
tokens = sentence_to_token_ids(sentence)

# Check if there's a 3-gram "ein", "richtige" and "Satz" in p_3gram
last_two_tokens = (tokens[0], tokens[1])
next_token = tokens[2]

p_3gram[last_two_tokens][next_token] if last_two_tokens in p_3gram and next_token in p_3gram[last_two_tokens] else 0	

0

Let's look at the conditional probabilities

In [107]:
last_two_tokens = (sentence_to_token_ids('ein')[0], sentence_to_token_ids('richtige')[0])
next_token = sentence_to_token_ids('Satz')[0]

if last_two_tokens in p_3gram and next_token in p_3gram[last_two_tokens]:
    prob = lambdas_3gram[last_two_tokens] * p_3gram[last_two_tokens][next_token] + (1 - lambdas_3gram[last_two_tokens]) * (lambdas_2gram[last_two_tokens[-1]] * p_2gram[last_two_tokens[-1]][next_token] + (1 - lambdas_2gram[last_two_tokens[-1]]) * p_1gram_smoothed[next_token])
else:
    prob = lambdas_2gram[last_two_tokens[-1]] * p_2gram[last_two_tokens[-1]][next_token] + (1 - lambdas_2gram[last_two_tokens[-1]]) * p_1gram_smoothed[next_token]

print ('p(Satz|ein, richtige)', prob)

p(Satz|ein, richtige) 6.403467638530913e-05


In [108]:
last_two_tokens = (sentence_to_token_ids('ein')[0], sentence_to_token_ids('richtiger')[0])
next_token = sentence_to_token_ids('Satz')[0]

if last_two_tokens in p_3gram and next_token in p_3gram[last_two_tokens]:
    prob = lambdas_3gram[last_two_tokens] * p_3gram[last_two_tokens][next_token] + (1 - lambdas_3gram[last_two_tokens]) * (lambdas_2gram[last_two_tokens[-1]] * p_2gram[last_two_tokens[-1]][next_token] + (1 - lambdas_2gram[last_two_tokens[-1]]) * p_1gram_smoothed[next_token])
else:
    prob = lambdas_2gram[last_two_tokens[-1]] * p_2gram[last_two_tokens[-1]][next_token] + (1 - lambdas_2gram[last_two_tokens[-1]]) * p_1gram_smoothed[next_token]

print ('p(Satz|ein, richtiger)', prob)

p(Satz|ein, richtiger) 0.0012552842078156863


### References
[1] Wolfer, Sascha, Koplenig, Alexander, Kupietz, Marc & Müller-Spitzer, Carolin. submitted. Introducing DeReKoGram.

[2] Brants, Thorsten & Popat, Ashok C. & Xu, Peng & Och, Franz J. & Dean, Jeffrey. 2007. Large language models in machine translation. In Proceedings of the 2007 joint conference on empirical methods in natural language processing and computational natural language learning (EMNLP-CoNLL), 858–867. Prague, Czech Republic: Association for Computational Linguistics. Retrieved from [https://aclanthology.org/D07-1090](https://aclanthology.org/D07-1090)

[3] Schmid, Helmut. 1994. Probabilistic Part-of-Speech Tagging Using Decision Trees. In International Conference on New Methods in Language Processing,. Manchester, UK. Retrieved from [https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger1](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/tree-tagger1.pdf)

[4] Wolfer, Sascha & Hein, Katrin. 2022. Konsequenzen der los-Suffigierung im Deutschen: Korpushäufigkeit, emotional-affektive Effekte und konstruktionsgrammatische Perspektiven. Zeitschrift für Wortbildung / Journal of Word Formation 6(2). 71–99. DOI: [https://doi.org/10.3726/zwjw.2022.02.03](https://doi.org/10.3726/zwjw.2022.02.03)  

[5] Stadler, Heike. 2014. Die Erstellung der Basislemmaliste der neuhochdeutschen Standardsprache aus mehrfach linguistisch annotierten Korpora. (H. Blühdorn & M. Elstermann & A. Klosa, Eds.). Mannheim: Institut für Deutsche Sprache. Retrieved from [https://nbn-resolving.org/urn:nbn:de:bsz:mh39-29999](https://nbn-resolving.org/urn:nbn:de:bsz:mh39-29999)

[6] Chen, Stanley F. & Joshua Goodman. 1996. An Empirical Study of Smoothing Techniques for Language Modeling. In 34th Annual Meeting of the Association for Computational Linguistics, 310–318. Santa Cruz, California, USA: Association for Computational Linguistics. https://doi.org/10.3115/981863.981904. [https://aclanthology.org/P96-1041](https://aclanthology.org/P96-1041).


**Disclaimer: I used `GitHub Copilot` and `ChatGPT` when producing this notebook.**